<a href="https://colab.research.google.com/github/Himagination/TSAI_self_learning/blob/main/Session5/Lecture_Session_TorchText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchtext.datasets import AG_NEWS

In [3]:
help(AG_NEWS)

Help on function AG_NEWS in module torchtext.datasets.ag_news:

AG_NEWS(root='.data', split=('train', 'test'))
    AG_NEWS dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 120000
    
        test: 7600
    
    
    Number of classes
        4
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [4]:
train_iter = AG_NEWS(split='train')

29.5MB [00:00, 99.4MB/s]


In [5]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [6]:
# Let's iterate through train_iter
for (line_number, (label, text)) in enumerate(train_iter):
  print(label, text)
  if line_number == 19:
    break

3 Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
3 Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
3 Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
3 Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.


In [8]:
from torch.utils.data import DataLoader
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, 
                        batch_size=8, 
                        shuffle=False)

In [9]:
next(iter(dataloader))

[tensor([3, 3, 3, 3, 3, 3, 3, 3]),
 ("Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
  'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
  "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
  'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
  'Oil prices soar to all-time re

In [10]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [11]:
help(get_tokenizer)

Help on function get_tokenizer in module torchtext.data.utils:

get_tokenizer(tokenizer, language='en')
    Generate tokenizer function for a string sentence.
    
    Args:
        tokenizer: the name of tokenizer function. If None, it returns split()
            function, which splits the string sentence by space.
            If basic_english, it returns _basic_english_normalize() function,
            which normalize the string first and split by space. If a callable
            function, it will return the function. If a tokenizer library
            (e.g. spacy, moses, toktok, revtok, subword), it returns the
            corresponding library.
        language: Default en
    
    Examples:
        >>> import torchtext
        >>> from torchtext.data import get_tokenizer
        >>> tokenizer = get_tokenizer("basic_english")
        >>> tokens = tokenizer("You can now install TorchText using pip!")
        >>> tokens
        >>> ['you', 'can', 'now', 'install', 'torchtext', 'using

In [12]:
help(build_vocab_from_iterator)

Help on function build_vocab_from_iterator in module torchtext.vocab.vocab_factory:

build_vocab_from_iterator(iterator: Iterable, min_freq: int = 1, specials: Union[List[str], NoneType] = None, special_first: bool = True) -> torchtext.vocab.vocab.Vocab
    Build a Vocab from an iterator.
    
    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        min_freq: The minimum frequency needed to include a token in the vocabulary.
        specials: Special symbols to add. The order of supplied tokens will be preserved.
        special_first: Indicates whether to insert symbols at the beginning or at the end.
    
    
    Returns:
        torchtext.vocab.Vocab: A `Vocab` object
    
    Examples:
        >>> #generating vocab from text file
        >>> import io
        >>> from torchtext.vocab import build_vocab_from_iterator
        >>> def yield_tokens(file_path):
        >>>     with io.open(file_path, encoding = 'utf-8') as f:
        >>> 

In [13]:
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
  for _, text in data_iter:
    yield(tokenizer(text))

In [14]:
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

In [15]:
vocab.set_default_index(vocab["<unk>"])

In [16]:
vocab(['here', 'is', 'an', 'example', 'of', 'alien', 'invasion', 'they', 'are', 'called', 'symphonites'])

[475, 21, 30, 5297, 6, 7838, 3042, 67, 42, 440, 0]

In [17]:
# Write a simple lambda function which takes the tokenizer and create a vocabulary
# We cannot pass a sentence directly to `vocab` as it expects a iterator.
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [18]:
text_pipeline("Here is an example of alien invasion, and they are called bangaloriters")

[475, 21, 30, 5297, 6, 7838, 3042, 3, 8, 67, 42, 440, 0]

In [19]:
import torch

def collate_fn(batch):
  src_batch, tgt_batch = [], []

  for src_batch, tgt_batch in batch:
    src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
    tgt_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip('\n')))
  src_batch = pad_sequences(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequences(tgt_batch, padding_value=PAD_IDX)
  return src_batch, tgt_batch

In [20]:
weight = torch.randn(3, 4)
weight

tensor([[-1.8973,  0.0233, -0.9136, -0.9915],
        [ 1.2545,  1.5623,  1.1443, -0.7207],
        [-0.1572,  1.3917, -0.3657, -0.6832]])

In [21]:
indices = torch.tensor([2, 1])
indices

tensor([2, 1])

In [23]:
embeddings = torch.nn.functional.embedding(indices, weight)
embeddings

tensor([[-0.1572,  1.3917, -0.3657, -0.6832],
        [ 1.2545,  1.5623,  1.1443, -0.7207]])

In [24]:
embeddings.mean(dim=0, keepdim=True)

tensor([[ 0.5487,  1.4770,  0.3893, -0.7020]])

In [25]:
torch.nn.functional.embedding_bag(indices, 
                                  weight, torch.tensor([0]), 
                                  mode="mean")

tensor([[ 0.5487,  1.4770,  0.3893, -0.7020]])

In [26]:
def collate_fn(batch):
  label_list, text_list, offsets = [], [], [0]
  for (_label, _text) in batch:
    label_list.append(label_pipeline(_label))
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))

  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)

In [29]:
train_iter = AG_NEWS(split="train")
dataloader = DataLoader(train_iter, 
                        batch_size=8, 
                        shuffle=False, 
                        collate_fn=collate_fn)

In [30]:
from torch import nn

class TextClassificationModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
    self.fc = nn.Linear(embed_dim, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  def forward(self, text, offsets):
    embedded = self.embedding(text, offsets)
    return self.fc(embedded)

**Labels Description**

1: World
2: Sports
3: Business
4: Sci/Tech

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter = AG_NEWS(split="train")
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emb_size = 64
model = TextClassificationModel(vocab_size, emb_size, num_class).to(device)

In [33]:
import time

def train(dataloader):
  model.train()
  total_acc, total_count = 0, 0
  log_interval = 500
  start_time = time.time()

  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time() - start_time
      print('| epoch {:3d} | {:5d}/{:5d} batches '
            '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader), 
                                        total_acc/total_count))
      total_acc, total_count = 0, 0
      start_time = time.time()

def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
  return total_acc/total_count

In [35]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10
LR = 5
BATCH_SIZE = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
      random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, 
                              shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, 
                              shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, 
                             shuffle=True, collate_fn=collate_fn)

for epoch in range(1, EPOCHS + 1):
  epoch_start_time = time.time()
  train(train_dataloader)
  accu_val = evaluate(valid_dataloader)
  if total_accu is not None and total_accu > accu_val:
    scheduler.step()
  else:
    total_accu = accu_val
  print('-' *59)
  print('| and of epoch {:3d} | time {:5.2f}s | '
        'valid accuracy {:8.3f} '.format(epoch, 
                                         time.time() - epoch_start_time, 
                                         accu_val))
  print('-'* 59)

1.86MB [00:00, 73.0MB/s]                  


| epoch   1 |   500/ 1782 batches | accuracy    0.678
| epoch   1 |  1000/ 1782 batches | accuracy    0.852
| epoch   1 |  1500/ 1782 batches | accuracy    0.876
-----------------------------------------------------------
| and of epoch   1 | time  7.31s | valid accuracy    0.884 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.898
| epoch   2 |  1000/ 1782 batches | accuracy    0.898
| epoch   2 |  1500/ 1782 batches | accuracy    0.901
-----------------------------------------------------------
| and of epoch   2 | time  7.15s | valid accuracy    0.900 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.915
| epoch   3 |  1000/ 1782 batches | accuracy    0.913
| epoch   3 |  1500/ 1782 batches | accuracy    0.916
-----------------------------------------------------------
| and of epoch   3 | time  7.09s | valid accuracy    0.899 
----------------------------------

In [36]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.905


In [37]:
ag_news_label = {1: "World", 
                 2: "Sports", 
                 3: "Business", 
                 4: "Sci/tech"}

def predict(text, text_pipeline):
  with torch.no_grad():
    text = torch.tensor(text_pipeline(text))
    output = model(text, torch.tensor([0]))
    return output.argmax(1).item() + 1